In [1]:
# !pip install wrds
# !pip install swig
# !pip install -q condacolab
# import condacolab
# condacolab.install()
# !apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-xia3ba8o
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-xia3ba8o
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit d57b7517150f1eb28a09b868f17aec5b6d2cc4fb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-0y_b5beh/elegantrl_4e5233d20eff4dc382b0b627fb894844
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-0y_b5beh/elegantrl_4e5233d20eff4dc382b0b627fb894844
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit a531d493b710a872a4943532872846e00ad96abf
  Preparing met

In [164]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

## Import packages

In [2]:

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline
import datetime

from finrl import config
from finrl import config_tickers
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
# from finrl.meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts
from finrl.meta.data_processor import DataProcessor
from finrl.meta.data_processors.processor_yahoofinance import YahooFinanceProcessor
import sys
sys.path.append("../FinRL-Library")

/home/jenchang/.local/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [3]:
# Prepare data
dividends = pd.read_csv('/home/jenchang/drl_finance/data/asset_Dividends.csv')
monetum = pd.read_csv('/home/jenchang/drl_finance/data/asset_monetum.csv')
# price = pd.read_csv('/home/jenchang/drl_finance/data/asset_price.csv')
sma = pd.read_csv('/home/jenchang/drl_finance/data/asset_sma.csv')
rate = pd.read_csv('/home/jenchang/drl_finance/data/bond_rate.csv')
fred = pd.read_csv('/home/jenchang/drl_finance/data/fred_data.csv')
fund = pd.read_csv('/home/jenchang/drl_finance/data/fundamentals.csv')
tips = pd.read_csv('/home/jenchang/drl_finance/data/TIPS_signal.csv')

# data =pd.DataFrame({'Date': dividends['Date']})
# data = data.merge(dividends, on='Date', how = 'left')
# data = data.merge(monetum, on='Date', how = 'left')
# # data = data.merge(price, on='Date', how = 'left')
# data = data.merge(sma, on='Date', how = 'left')
# data = data.merge(rate, on='Date', how = 'left')
# data = data.merge(fred, on='Date', how = 'left')
# data = data.merge(fund, on='Date', how = 'left')
# data = data.merge(tips, on='Date', how = 'left')



In [4]:
dividends_pivots = pd.DataFrame()
stock_list = ["SPY","AGG","QQQ","VNQ","IWM","VIG","IEF"]
for stock in stock_list:
    temp_df = dividends[['Date', f'Dividends_{stock}', f'Dividends_growth_{stock}']]
    temp_df = temp_df.rename(columns = {f'Dividends_{stock}': 'Dividends', f'Dividends_growth_{stock}':'Dividends_growth'})
    temp_df['tic'] = stock

    dividends_pivots = pd.concat([dividends_pivots, temp_df])


In [5]:
monetum_pivots = pd.DataFrame()
stock_list = ["SPY","AGG","QQQ","VNQ","IWM","VIG","IEF"]
for stock in stock_list:
    temp_df = monetum[['Date', f'{stock}_20', f'{stock}_60', f'{stock}_120', f'{stock}_180', f'{stock}_240', f'{stock}_sum', f'{stock}_avg']]
    temp_df = temp_df.rename(columns = {f'{stock}_20': 'monetum_20', f'{stock}_40': 'monetum_40', f'{stock}_60': 'monetum_60', f'{stock}_120': 'monetum_120', f'{stock}_180': 'monetum_180',f'{stock}_240': 'monetum_240', f'{stock}_sum': 'monetum_sum', f'{stock}_avg': 'monetum_avg'})
    temp_df['tic'] = stock

    monetum_pivots = pd.concat([monetum_pivots, temp_df])   


In [6]:
sma_pivots = pd.DataFrame()
stock_list = ["SPY","AGG","QQQ","VNQ","IWM","VIG","IEF"]
for stock in stock_list:
    temp_df = sma[['Date', f'{stock}_close_20_sma', f'{stock}_close_50_sma', f'{stock}_close_100_sma']]
    temp_df = temp_df.rename(columns = {f'{stock}_close_20_sma': 'sma_20', f'{stock}_close_50_sma': 'sma_50', f'{stock}_close_100_sma': 'sma_100'})
    temp_df['tic'] = stock

    sma_pivots = pd.concat([sma_pivots, temp_df])   

In [7]:
import datetime
import yfinance as yf
from datetime import date
from datetime import timedelta
stock_list = ["SPY","AGG","QQQ","VNQ","IWM","VIG","IEF"]
start_date = '2007-03-01'
end_date = '2024-03-31'
# end_date = '2024-03-31'
start_date = pd.Timestamp(start_date)
end_date = pd.Timestamp(end_date)
delta = timedelta(days=1)
data_df = pd.DataFrame()
for tic in stock_list:

    # while (
    #     start_date <= end_date
    # ):  # downloading daily to workaround yfinance only allowing  max 7 calendar (not trading) days of 1 min data per single download
    temp_df = yf.download(
        tic,
        start=start_date,
        end=end_date,
        interval='1d',
        proxy=None,
    )
    temp_df["tic"] = tic
    data_df = pd.concat([data_df, temp_df])
        # start_date += delta

data_df = data_df.reset_index().drop(columns=["Adj Close"])
# convert the column names to match processor_alpaca.py as far as poss
data_df.columns = [
    "timestamp",
    "open",
    "high",
    "low",
    "close",
    "volume",
    "tic",
]
data_df = data_df.rename(columns={'timestamp': 'Date'})

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [8]:
dividends_pivots['Date'] = pd.to_datetime(dividends_pivots['Date'])
data_df = data_df.merge(dividends_pivots, on=['Date','tic'], how = 'left')

monetum_pivots['Date'] = pd.to_datetime(monetum_pivots['Date'])
data_df = data_df.merge(monetum_pivots, on=['Date','tic'], how = 'left')

sma_pivots['Date'] = pd.to_datetime(sma_pivots['Date'])
data_df = data_df.merge(sma_pivots, on=['Date','tic'], how = 'left')

rate.fillna(method = 'ffill', inplace = True)
rate['Date'] = pd.to_datetime(rate['Date'])
data_df = data_df.merge(rate, on=['Date'], how = 'left')

fred['Date'] = pd.to_datetime(fred['Date'])
data_df = data_df.merge(fred, on=['Date'], how = 'left')

fund['Date'] = pd.to_datetime(fund['Date'])
data_df = data_df.merge(fund, on=['Date'], how = 'left')

tips['Date'] = pd.to_datetime(tips['Date'])
data_df = data_df.merge(tips, on=['Date'], how = 'left')




/tmp/ipykernel_509664/2140797064.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  rate.fillna(method = 'ffill', inplace = True)


#Preprocess data
##Add covariance matrix as states

In [9]:
# # add covariance matrix as states
data_df=data_df.sort_values(['Date','tic'],ignore_index=True)
data_df.index = data_df.Date.factorize()[0]

cov_list = []
return_list = []

# # look back is one year
lookback= 2
for i in range(lookback,len(data_df.index.unique())):
  data_lookback = data_df.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'Date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  return_list.append(return_lookback)

  covs = return_lookback.cov().values 
  cov_list.append(covs)

  
df_cov = pd.DataFrame({'Date':data_df.Date.unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
data_df = data_df.merge(df_cov, on='Date')
data_df = data_df.sort_values(['Date','tic']).reset_index(drop=True)

#Training data split : 2007-03-01 to 2020-03-31


In [10]:
data_df = data_df.rename(columns ={'Date': 'date'})
train = data_split(data_df, '2007-03-01','2020-03-31')

#Build environment for portfolio allocation


In [15]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv


class StockPortfolioEnv(gym.Env):
    """A single stock trading environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then 
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step
        

    """
    metadata = {'render.modes': ['human']}

    def __init__(self, 
                df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                turbulence_threshold=None,
                lookback=252,
                day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,)) 
        # Shape = (34, 30)
        # covariance matrix + technical indicators
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False     
        self.turbulence_threshold = turbulence_threshold        
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

        
    def step(self, actions):
        # print(self.day)
        self.terminal = self.day >= len(self.df.index.unique())-1
        # print(actions)

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(),'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()
            
            plt.plot(self.portfolio_return_memory,'r')
            plt.savefig('results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))           
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() !=0:
              sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                       df_daily_return['daily_return'].std()
              print("Sharpe: ",sharpe)
            print("=================================")
            
            return self.state, self.reward, self.terminal,{}

        else:
            #print("Model actions: ",actions)
            # actions are the portfolio weight
            # normalize to sum of 1
            #if (np.array(actions) - np.array(actions).min()).sum() != 0:
            #  norm_actions = (np.array(actions) - np.array(actions).min()) / (np.array(actions) - np.array(actions).min()).sum()
            #else:
            #  norm_actions = actions
            weights = self.softmax_normalization(actions) 
            #print("Normalized actions: ", weights)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
            #print(self.state)
            # calcualte portfolio return
            # individual stocks' return * weight
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values)-1)*weights)
            dividends_growth = sum(self.data.Dividends_growth.values*weights)
            monetum = sum(self.data.monetum_avg*weights)
            
            # update portfolio value
            new_portfolio_value = self.portfolio_value*(1+portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])            
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value 
            #print("Step reward: ", self.reward)
            #self.reward = self.reward*self.reward_scaling

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False 
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]] 
        return self.state
    
    def render(self, mode='human'):
        return self.state
        
    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        return softmax_output

    
    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']
        
        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [12]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 7, State Space: 7


In [16]:
indicator_list =['Dividends',
       'Dividends_growth', 'monetum_20', 'monetum_60', 'monetum_120',
       'monetum_180', 'monetum_240', 'monetum_sum', 'monetum_avg', 'sma_20',
       'sma_50', 'sma_100', 'Close_US1M', 'Close_US3M', 'Close_DGS5',
       'Close_DGS10', 'DFF', 'CPIAUCSL', 'CSUSHPINSA', 'CPIAUCSL_PC1',
       'CPILFESL_PC1', 'GDP', 'GDPC1', '^VIX', '^GSPC', '^DJI', '^IXIC',
       '^RUT', '^W5000FLT', 'GC=F', 'CL=F', 'Bull-Bear Spread',
       'TIPS_adj_close', 'TIPS_240', 'TIPS_momentum']
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "transaction_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": indicator_list, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)

In [18]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


/home/jenchang/.local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [46]:
e_train_gym.df.loc[1].close.values

array([100.51999664,  83.48999786,  77.22000122,  42.84999847,
       139.69999695,  53.27000046,  79.11000061])

In [58]:
e_train_gym.df.loc[0].TIPS_momentum

0    1.717576
0    1.717576
0    1.717576
0    1.717576
0    1.717576
0    1.717576
0    1.717576
Name: TIPS_momentum, dtype: float64

In [51]:
sum((e_train_gym.df.loc[1].close.values / e_train_gym.df.loc[0].close.values)-1)

0.10571841436909057

#Implement DRL Algorithms

In [51]:
# initialize
agent = DRLAgent(env = env_train)
DDPG_PARAMS = {"batch_size": 128, "buffer_size": 50000, "learning_rate": 0.001}
model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cuda device


In [52]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000)

begin_total_asset:1000000
end_total_asset:2127864.3202056778
Sharpe:  0.4243379353283869
begin_total_asset:1000000
end_total_asset:2220118.0056861606
Sharpe:  0.44318791070904984
begin_total_asset:1000000
end_total_asset:2220118.0056861606
Sharpe:  0.44318791070904984
begin_total_asset:1000000
end_total_asset:2220118.0056861606
Sharpe:  0.44318791070904984
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 106       |
|    time_elapsed    | 123       |
|    total_timesteps | 13168     |
| train/             |           |
|    actor_loss      | -6.71e+07 |
|    critic_loss     | 7.82e+11  |
|    learning_rate   | 0.001     |
|    n_updates       | 13067     |
|    reward          | 2220118.0 |
----------------------------------
begin_total_asset:1000000
end_total_asset:2220118.0056861606
Sharpe:  0.44318791070904984
begin_total_asset:1000000
end_total_asset:2220118.0056861606
Sharpe:  0.44318791070904984
begin_

In [54]:
trained_ddpg.save('/home/jenchang/drl_finance/trained_models/trained_ddpg.zip')

#Trading

In [20]:
trade = data_split(data_df,'2020-04-01', '2024-03-31')
trade['^W5000FLT'] = trade['^W5000FLT'].str.replace(',', '')
e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)

In [125]:
df_daily_return, df_actions = DRLAgent.DRL_prediction(model=trained_ddpg,
                        environment = e_trade_gym)

/home/jenchang/.local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


begin_total_asset:1000000
end_total_asset:1769578.5850675697
Sharpe:  0.9849634105680612
hit end!


#Backtest 

In [205]:
from pyfolio import timeseries
DRL_strat = convert_daily_return_to_pyfolio_ts(df_daily_return)
perf_func = timeseries.perf_stats 
perf_stats_all = perf_func( returns=DRL_strat, 
                              factor_returns=DRL_strat, 
                                positions=None, transactions=None, turnover_denom="AGB")

In [206]:
perf_stats_all

Annual return          0.153858
Cumulative returns     0.769579
Annual volatility      0.157995
Sharpe ratio           0.984963
Calmar ratio           0.598755
Stability              0.261677
Max drawdown          -0.256964
Omega ratio            1.182565
Sortino ratio          1.440469
Skew                  -0.033654
Kurtosis               2.751562
Tail ratio             1.059336
Daily value at risk   -0.019288
Alpha                  0.000000
Beta                   1.000000
dtype: float64

#60:40 投資策略 、 等比投資策略

In [153]:
 
stocks = ["QQQ","VNQ","IWM","VIG"]
bonds = ["AGG","IEF"]

start_date = '2020-04-01'
end_date = '2024-03-31'

#download data
data = yf.download(stocks + bonds, start="2020-03-01", end="2024-03-31")['Adj Close']

#caculate daily return 
daily_returns = data.pct_change()

[*********************100%%**********************]  6 of 6 completed


In [212]:
# 60:40

stock_ratio = 0.6
bond_ratio = 0.4

stock_returns = daily_returns[stocks].mean(axis=1) * stock_ratio
bond_returns = daily_returns[bonds].mean(axis=1) * bond_ratio
portfolio_daily_returns = stock_returns + bond_returns
portfolio_daily_returns = portfolio_daily_returns.fillna(0)
portfolio_daily_returns= portfolio_daily_returns.reset_index()
portfolio_daily_returns = portfolio_daily_returns.rename(columns = {'Date': 'date', 0: 'daily_return'})

#計算綜合評量結果
set6040_strat = convert_daily_return_to_pyfolio_ts(portfolio_daily_returns)
perf_func = timeseries.perf_stats 
perf_stats_all_6040 = perf_func( returns=set6040_strat, 
                              factor_returns=set6040_strat, 
                                positions=None, transactions=None, turnover_denom="AGB")

In [213]:
perf_stats_all_6040

Annual return           0.065803
Cumulative returns      0.296565
Annual volatility       0.146951
Sharpe ratio            0.507444
Calmar ratio            0.286371
Stability               0.143216
Max drawdown           -0.229782
Omega ratio             1.098208
Sortino ratio           0.704139
Skew                   -0.654741
Kurtosis               10.043646
Tail ratio              0.989042
Daily value at risk    -0.018218
Alpha                   0.000000
Beta                    1.000000
dtype: float64

In [209]:
#等比投資
portfolio_daily_returns = daily_returns.mean(axis=1)
portfolio_daily_returns = portfolio_daily_returns.fillna(0)
portfolio_daily_returns= portfolio_daily_returns.reset_index()
portfolio_daily_returns = portfolio_daily_returns.rename(columns = {'Date': 'date', 0: 'daily_return'})

#計算綜合評量結果
avg_strat = convert_daily_return_to_pyfolio_ts(portfolio_daily_returns)
perf_func = timeseries.perf_stats 
perf_stats_all_avg = perf_func( returns=avg_strat, 
                              factor_returns=avg_strat, 
                                positions=None, transactions=None, turnover_denom="AGB")

In [210]:
perf_stats_all_avg

Annual return           0.075162
Cumulative returns      0.343598
Annual volatility       0.161059
Sharpe ratio            0.530887
Calmar ratio            0.316422
Stability               0.194813
Max drawdown           -0.237538
Omega ratio             1.103221
Sortino ratio           0.736988
Skew                   -0.685012
Kurtosis               10.517597
Tail ratio              0.998947
Daily value at risk    -0.019952
Alpha                   0.000000
Beta                    1.000000
dtype: float64